In [1]:
import cv2
import numpy as np
import os
import warnings

import DetectChars
import DetectPlates
import PossiblePlate

In [2]:
#Constants with RGB values for further use
SCALAR_BLACK = (0.0, 0.0, 0.0)
SCALAR_WHITE = (255.0, 255.0, 255.0)
SCALAR_YELLOW = (0.0, 255.0, 255.0)
SCALAR_GREEN = (0.0, 255.0, 0.0)
SCALAR_RED = (0.0, 0.0, 255.0)

In [3]:
#Main Function
def main():
    #Attempt KNN training
    blnKNNTrainingSuccessful = DetectChars.loadKNNDataAndTrainKNN()         
    #If KNN training was not successful
    if blnKNNTrainingSuccessful == False:                               
        print("\nerror: KNN traning was not successful\n")  
        return  
    #Reading an image
    imgOriginalScene  = cv2.imread("10.png")               
    if imgOriginalScene is None:                            
        print("\nError: Image not read")                                    
        return                                              
    #Calling DertectPlates function to detect plates
    listOfPossiblePlates = DetectPlates.detectPlatesInScene(imgOriginalScene)           
    #Calling DetectChars function to detect characters on plates
    listOfPossiblePlates = DetectChars.detectCharsInPlates(listOfPossiblePlates)        
    #If no plates were found
    if len(listOfPossiblePlates) == 0:                          
        print("\nNo license plates were detected\n")  
    else:                                                       
        #Sort the list of possible plates in DESCENDING order (most number of chars to least number of chars)
        listOfPossiblePlates.sort(key = lambda possiblePlate: len(possiblePlate.strChars), reverse = True)
        #Suppose the plate with the most recognized chars (the first plate in sorted by string length descending order) is the actual plate
        licPlate = listOfPossiblePlates[0]
        #If no characters were found in the plates
        if len(licPlate.strChars) == 0:                    
            print("\nNo characters were detected") 
            return                                          
        #Draw red rectangle around plate
        drawRedRectangleAroundPlate(imgOriginalScene, licPlate)             
        print("\nLicense plate read from image = " + licPlate.strChars + "\n")  
        #To write license plate text on the image
        writeLicensePlateCharsOnImage(imgOriginalScene, licPlate)           
        cv2.imshow("License plate detected and read!", imgOriginalScene)                
        cv2.imwrite("imgOriginalScene.png", imgOriginalScene)           
    cv2.waitKey(0)
    return

In [4]:
#Fucntion to draw 4 vertices of rotated rectangle
def drawRedRectangleAroundPlate(imgOriginalScene, licPlate):
    p2fRectPoints = cv2.boxPoints(licPlate.rrLocationOfPlateInScene)           
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[0]), tuple(p2fRectPoints[1]), SCALAR_RED, 2)         
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[1]), tuple(p2fRectPoints[2]), SCALAR_RED, 2)
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[2]), tuple(p2fRectPoints[3]), SCALAR_RED, 2)
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[3]), tuple(p2fRectPoints[0]), SCALAR_RED, 2)

In [ ]:
#Function to write the characters read on the image
def writeLicensePlateCharsOnImage(imgOriginalScene, licPlate):
    #This will be the center of the area the text will be written to
    ptCenterOfTextAreaX = 0                             
    ptCenterOfTextAreaY = 0
    #This will be the bottom left of the area that the text will be written to
    ptLowerLeftTextOriginX = 0                          
    ptLowerLeftTextOriginY = 0
    sceneHeight, sceneWidth, sceneNumChannels = imgOriginalScene.shape
    plateHeight, plateWidth, plateNumChannels = licPlate.imgPlate.shape
    intFontFace = cv2.FONT_HERSHEY_SIMPLEX                      
    fltFontScale = float(plateHeight) / 30.0                    
    intFontThickness = int(round(fltFontScale * 1.5))           
    textSize, baseline = cv2.getTextSize(licPlate.strChars, intFontFace, fltFontScale, intFontThickness)        
    #Unpack rotated rectangle into center point, width and height, and angle
    ( (intPlateCenterX, intPlateCenterY), (intPlateWidth, intPlateHeight), fltCorrectionAngleInDeg ) = licPlate.rrLocationOfPlateInScene
    #Centers
    intPlateCenterX = int(intPlateCenterX)              
    intPlateCenterY = int(intPlateCenterY)
    #The horizontal location of the text area is the same as the length of plate
    ptCenterOfTextAreaX = int(intPlateCenterX)         
    #If the license plate is in the upper 3/4 of the image
    if intPlateCenterY < (sceneHeight * 0.75):
        #To write the chars in below the plate
        ptCenterOfTextAreaY = int(round(intPlateCenterY)) + int(round(plateHeight * 1.6))
    #Else, if the license plate is in the lower 1/4 of the image
    else:                                      
        #To write the chars in above the plate
        ptCenterOfTextAreaY = int(round(intPlateCenterY)) - int(round(plateHeight * 1.6))      
    textSizeWidth, textSizeHeight = textSize                
    #To calculate the lower left origin of the text area based on the text area center, width, and height
    ptLowerLeftTextOriginX = int(ptCenterOfTextAreaX - (textSizeWidth / 2))           
    ptLowerLeftTextOriginY = int(ptCenterOfTextAreaY + (textSizeHeight / 2))          
    #Write the text on the image
    cv2.putText(imgOriginalScene, licPlate.strChars, (ptLowerLeftTextOriginX, ptLowerLeftTextOriginY), intFontFace, fltFontScale, SCALAR_YELLOW, intFontThickness)

if __name__ == "__main__":
    warnings.filterwarnings("ignore", category=DeprecationWarning) 
    main()

Car Z00MN65 has entered the parking lot at 2021-04-25 22:36:27.993978
The Car Z00MN65 has exited the parking lot after 1.0002495166666667 minutes
Fare is 50.01 Rs

License plate read from image = Z00MN65

